In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math
from datetime import datetime, timedelta
from scipy import stats

# Function to comvert the "HIRE DATE1" column into usable date format
def from_excel_ordinal(ordinal, epoch=datetime(1900, 1, 1)):
    # Adapted from above, thanks to @Martijn Pieters 

    if ordinal > 59:
        ordinal -= 1  # Excel leap year bug, 1900 is not a leap year!
    inDays = int(ordinal)
    frac = ordinal - inDays
    inSecs = int(round(frac * 86400.0))

    return epoch + timedelta(days=inDays - 1, seconds=inSecs) # epoch is day 1

last_date = "2016-12-31"

datetime.strptime(last_date, '%Y-%m-%d')

datetime.datetime(2016, 12, 31, 0, 0)

In [2]:
city_data = pd.read_excel("FY16 SA City Compensation.xlsx")
city_data["HIRE DATE1"] = city_data["HIRE DATE1"].apply(from_excel_ordinal)

In [3]:
city_data.head()

,REF #,FIRST NAME,MIDDLE NAME,LAST NAME,HIRE DATE1,FY16 ANNUAL SALARY2,FY16 BASE PAY3,FY16 LEAVE PAYOUT4,FY16 OTHER5,FY16 OVERTIME6,FY16 GROSS EARNINGS7,FY16 ADDITIONAL BENEFITS8,FY16 TOTAL COMPENSATION9,JOB TITLE,BUSINESS AREA,GENDER,ETHNIC ORIGIN10,EMPLOYEE SUBGROUP
0,1,SHERYL,L.,SCULLEY,2005-11-07,425000.00,414615.38,0.00,97354.89,0.0,511970.27,75379.480000,587349.750000,1022-CITY MANAGER,CITY MANAGER'S OFFICE,FEMALE,WHITE,2A CITY MGR FT
1,2,CASANDRA,BROWN,MATEJ,2011-06-13,200906.16,196106.64,12751.20,46221.88,0.0,255079.72,49355.342435,304435.062435,1013-CVB DIRECTOR,CONVENTION AND VISITORS BUREAU,FEMALE,WHITE,2B DEPT DIR FT
2,3,PETER,DAMIAN,ZANONI,1997-04-02,230405.76,219271.20,8112.80,6840.00,0.0,234224.00,48466.712147,282690.712147,1021-DEPUTY CITY MANAGER,CITY MANAGER'S OFFICE,MALE,WHITE,2B DEPT DIR FT
3,4,ERIK,J,WALSH,1994-06-06,235014.26,223656.70,0.00,6840.00,0.0,230496.70,49268.050267,279764.750267,1021-DEPUTY CITY MANAGER,CITY MANAGER'S OFFICE,MALE,HISPANIC,2B DEPT DIR FT
4,5,CHARLES,N.,HOOD,2007-04-16,202562.62,197723.57,12397.05,9480.00,0.0,219600.62,46434.995086,266035.615086,1003-FIRE CHIEF,FIRE,MALE,BLACK,2C F&P CHIEF FT


> How does gender and compensation compare?

* $H_0$: The average pay for both genders is the same
* $H_a$: The average pay for both genders is not the same

In [33]:
x1 = city_data[city_data["GENDER"] == "MALE"]["FY16 TOTAL COMPENSATION9"]
x2 = city_data[city_data["GENDER"] == "FEMALE"]["FY16 TOTAL COMPENSATION9"]

In [34]:
stats.ttest_ind(x1, x2)

Ttest_indResult(statistic=41.409566842956735, pvalue=0.0)

In [37]:
print(f'''
Becasue our p value {p:.3} is smaller than our alpha, we reject our null hypothesis, and thus conclude there is a significance in mean 
compensation between gender
''')


Becasue our p value 0.0 is smaller than our alpha, we reject our null hypothesis, and thus conclude there is a significance in mean 
compensation between gender



> Does one gender have a higher mean than the other? 

* $H_0$: The mean difference in mean between gender and total population is the same
* $H_a$: The mean difference between gender and total population is not the same

In [41]:
x_male = city_data[city_data["GENDER"] == "MALE"]["FY16 TOTAL COMPENSATION9"]
mu = city_data["FY16 TOTAL COMPENSATION9"].mean()

t_stat_male, p_male = stats.ttest_1samp(x_male, mu)

t_stat_male, p_male

(22.628992838960947, 7.412691481808962e-110)

In [42]:
x_female = city_data[city_data["GENDER"] == "FEMALE"]["FY16 TOTAL COMPENSATION9"]
mu = city_data["FY16 TOTAL COMPENSATION9"].mean()

t_stat_female, p_female = stats.ttest_1samp(x_female, mu)

t_stat_female, p_female

(-40.14170823964582, 7.36730972379532e-298)

Because our t-test value for male is so much higher than female, we know that 

> Is the number of days an employee worked related to their full compensation? 

* $H_0$: There is no linear correlation between pay and tenure
* $H_a$: There is a linear correlation between pay and tenure

In [15]:
city_data = city_data.assign(length_time = datetime.strptime(last_date, '%Y-%m-%d') - city_data["HIRE DATE1"])

(datetime.strptime(last_date, '%Y-%m-%d') - city_data["HIRE DATE1"][0])/np.timedelta64(1, 'Y')

city_data.head()

,REF #,FIRST NAME,MIDDLE NAME,LAST NAME,HIRE DATE1,FY16 ANNUAL SALARY2,FY16 BASE PAY3,FY16 LEAVE PAYOUT4,FY16 OTHER5,FY16 OVERTIME6,FY16 GROSS EARNINGS7,FY16 ADDITIONAL BENEFITS8,FY16 TOTAL COMPENSATION9,JOB TITLE,BUSINESS AREA,GENDER,ETHNIC ORIGIN10,EMPLOYEE SUBGROUP,length_time
0,1,SHERYL,L.,SCULLEY,2005-11-07,425000.00,414615.38,0.00,97354.89,0.0,511970.27,75379.480000,587349.750000,1022-CITY MANAGER,CITY MANAGER'S OFFICE,FEMALE,WHITE,2A CITY MGR FT,4072 days
1,2,CASANDRA,BROWN,MATEJ,2011-06-13,200906.16,196106.64,12751.20,46221.88,0.0,255079.72,49355.342435,304435.062435,1013-CVB DIRECTOR,CONVENTION AND VISITORS BUREAU,FEMALE,WHITE,2B DEPT DIR FT,2028 days
2,3,PETER,DAMIAN,ZANONI,1997-04-02,230405.76,219271.20,8112.80,6840.00,0.0,234224.00,48466.712147,282690.712147,1021-DEPUTY CITY MANAGER,CITY MANAGER'S OFFICE,MALE,WHITE,2B DEPT DIR FT,7213 days
3,4,ERIK,J,WALSH,1994-06-06,235014.26,223656.70,0.00,6840.00,0.0,230496.70,49268.050267,279764.750267,1021-DEPUTY CITY MANAGER,CITY MANAGER'S OFFICE,MALE,HISPANIC,2B DEPT DIR FT,8244 days
4,5,CHARLES,N.,HOOD,2007-04-16,202562.62,197723.57,12397.05,9480.00,0.0,219600.62,46434.995086,266035.615086,1003-FIRE CHIEF,FIRE,MALE,BLACK,2C F&P CHIEF FT,3547 days


In [24]:
x = city_data["FY16 TOTAL COMPENSATION9"] 
y = city_data["length_time"] / pd.to_timedelta(1, unit='D')

corr, p = stats.pearsonr(x,y)

print(f"corr = {corr}")
print(f" p = {p:.10f}")

corr = 0.5365723672372791
 p = 0.0000000000


We can reject our null hypothesis that full compensation and tenure are not linearly correlated. We can also see that there is a positive linear correlation between full compensation and tenure, however, because our correlation is ~0.54, it is not a very strong correlation and there might be other factors influencing this relationship.
